In [8]:
import sys
import time
sys.path.append("../Common/Crawler")
sys.path.append("../Common/Util")

import mod_crawler_base as craw

isLinux = True
isHidden = True

from time import sleep

wd = craw.InitWebDriver( isLinux, isHidden )

In [9]:


listSectionNm    = ['종합', '정치', '경제', '사회', '생활/문화', '세계', 'IT/과학', '포토', 'TV']
listSectionID    = ['100', '101', '102', '103', '104', '105', '106', '003', '115']
listSections     = []
listSections_URL = []

delay_sleep = 0.5



def get_naver_section_url( wd , ymd , sectionID ) :
    urlSect = "https://news.naver.com/main/ranking/popularDay.nhn?rankingType=popular_day&sectionId=%s&date=%s" % ( sectionID, ymd )
    craw.OpenWebPage(wd, urlSect ,1)
    listElem = wd.find_elements_by_class_name( 'ranking_item' )
    for i,elem in enumerate( listElem ):
        listSections.append( elem.text.split("\n")[0] )
        listSections_URL.append( elem.find_element_by_tag_name('a').get_attribute('href')  )
    return

def crawling_naver_sections(wd, ymd):
    css_head = "tts_head"
    css_contents = "article_body"
    for sectionID in listSectionID:
        get_naver_section_url(wd, ymd, sectionID)
        now = time.localtime()
        time_str = "%4d%02d%02d_%02d%02d%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
        time_str_day = "%4d%02d%02d" % (now.tm_year, now.tm_mon, now.tm_mday)
        with open("naver_section_" + sectionID + "_" + time_str + ".txt", "w+", encoding='utf-8') as f:
            idx = 0
            for url in listSections_URL:
                craw.OpenWebPage(wd, url, 1)
                # 1.TODAY_INDEX
                f.write(str(idx))
                f.write("\t")

                idx = idx + 1

                # 2.time_str_day
                f.write(time_str_day)
                f.write("\t")

                # 3.TITLE
                elem = wd.find_elements_by_class_name(css_head)
                print(elem[0].text)
                f.write(elem[0].text)
                f.write("\t")

                # 4.CONTENT
                elem = wd.find_elements_by_class_name(css_contents)
                content_txt = elem[0].text.replace("\n", "  ").replace("\r", "  ")
                
                # Naver 좋아요 이후로 제거 
                pos1 = content_txt.find( "좋아요" )
                pos2 = content_txt.find("훈훈해요")

                if( (pos2 - pos1) < 20 ) :
                    content_txt = content_txt[:pos1]

                print(content_txt)
                f.write(content_txt)
                f.write("\n")
                sleep( delay_sleep )


In [ ]:
def Main() :
    crawling_naver_sections(wd, "20180724")
Main()